In [74]:
import random
import warnings
import tensorflow as tf
from functions import *

warnings.simplefilter("ignore")

SEED_VALUE = 42
np.random.seed(SEED_VALUE)
random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

ticker = "AAPL"
dataset = load_data(ticker)
dataset, data = clean_data(dataset)
train_data, test_data, scaler = split_data(data)

# Define sequence length
sequence_length = 20

# Create sequences for LSTM models
X_train, y_train = create_dataset(train_data, sequence_length)
X_test, y_test = create_dataset(test_data, sequence_length)

# Build and train the LSTM model with best hyperparameters using the entire training data
best_params = get_best_params(X_train, y_train, tf, sequence_length, scaler)
lstm_model = build_lstm_model(tf, sequence_length, lr=best_params['learning_rate'], initializer=best_params['initializer'], loss_function=best_params['loss_function'])
lstm_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)

# Generate predictions for the LSTM model
lstm_predictions = lstm_model.predict(X_test)

# Generate naive predictions (using the last value in the training set)
naive_predictions = np.full_like(y_test, y_train[-1])

print_metrics(y_test, lstm_predictions, naive_predictions)

lstm_predictions_list = lstm_predictions.flatten().tolist()

one_year_data, one_year_data_2d, lstm_predictions_one_year = get_one_year_data(dataset, sequence_length, scaler, lstm_model)

print_trading_result(one_year_data, one_year_data_2d, lstm_predictions_one_year, sequence_length)

plot_trading_result(one_year_data, one_year_data_2d, lstm_predictions_one_year, sequence_length, ticker, "LSTM")

[*********************100%***********************]  1 of 1 completed


NameError: name 'split_data' is not defined